In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from loguru import logger
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

from housing_price_neural_net.config import MODELS_DIR
from housing_price_neural_net.modeling.train import HousePriceNet

val_data = np.load(MODELS_DIR / "val_data.npz")
X_val = val_data['X_val']
y_val = val_data['y_val']

X_val = torch.FloatTensor(X_val)
y_val = torch.LongTensor(y_val)

val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HousePriceNet(input_size=X_val.shape[1]).to(device)
model.load_state_dict(torch.load(MODELS_DIR / "model.pth"))
model.eval()

y_pred = []
y_true = []

with torch.no_grad():
    for batch_X, batch_y in val_loader:
        batch_X = batch_X.to(device)
        outputs = model(batch_X)
        _, predicted = torch.max(outputs.data, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(batch_y.numpy())

y_pred = np.array(y_pred)
y_true = np.array(y_true)

print(classification_report(y_true, y_pred, target_names=['cheap', 'average', 'expensive']))

plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['cheap', 'average', 'expensive'],
            yticklabels=['cheap', 'average', 'expensive'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

for i, class_name in enumerate(['cheap', 'average', 'expensive']):
    class_mask = y_true == i
    class_acc = np.mean(y_pred[class_mask] == y_true[class_mask])
    print(f"{class_name} accuracy: {class_acc:.4f}")

2025-03-22 23:35:29.628 | INFO     | housing_price_neural_net.config:<module>:11 - PROJ_ROOT path is: /home/bilski/uni_repos/HousingPriceNeuralNet


FileNotFoundError: [Errno 2] No such file or directory: '/home/bilski/uni_repos/HousingPriceNeuralNet/models/val_data.npz'